In [72]:
from music21 import *
import os
import matplotlib.pyplot as plt
import numpy as np

In [43]:
audio_teste = os.getcwd() + '/midi_songs_2/Aladdin.mid'
audio_teste

'/Users/luizeduardocartolano/Dropbox/DUDU/Unicamp/IC/MC906/workspace/Audio/midi_songs_2/Aladdin.mid'

[1,2,4,5,7,8,9,15,18,24,31,34]

In [50]:
def open_midi(midi_path, remove_drums):
    # There is an one-line method to read MIDIs
    # but to remove the drums we need to manipulate some
    # low level MIDI events.
    mf = midi.MidiFile()
    mf.open(midi_path)
    mf.read()
    mf.close()
    if (remove_drums):
        for i in range(len(mf.tracks)):
            mf.tracks[i].events = [ev for ev in mf.tracks[i].events if ev.channel != 10]          

    return midi.translate.midiFileToStream(mf)

In [51]:
base_midi = open_midi(audio_teste, True)
base_midi

<music21.stream.Score 0xa1b5fd080>

In [52]:
def list_instruments(midi):
    partStream = midi.parts.stream()
    print("List of instruments found on MIDI file:")
    for p in partStream:
        aux = p
        print (p.partName)

In [54]:
def extract_notes(midi_part):
    parent_element = []
    ret = []
    for nt in midi_part.flat.notes:        
        if isinstance(nt, note.Note):
            ret.append(max(0.0, nt.pitch.ps))
            parent_element.append(nt)
        elif isinstance(nt, chord.Chord):
            for pitch in nt.pitches:
                ret.append(max(0.0, pitch.ps))
                parent_element.append(nt)
    
    return ret, parent_element

In [57]:
timeSignature = base_midi.getTimeSignatures()[0]
music_analysis = base_midi.analyze('key')

In [59]:
print("Music time signature: {0}/{1}".format(timeSignature.beatCount, timeSignature.denominator))

Music time signature: 4/4


In [60]:
print("Expected music key: {0}".format(music_analysis))

Expected music key: d minor


In [61]:
print("Music key confidence: {0}".format(music_analysis.correlationCoefficient))

Music key confidence: 0.9012403644434548


In [62]:
print("Other music key alternatives:")
for analysis in music_analysis.alternateInterpretations:
    if (analysis.correlationCoefficient > 0.5):
        print(analysis)

Other music key alternatives:
a minor
F major
C major
G major
g minor
D major


In [64]:
import glob
names= glob.glob(os.getcwd()+"/midi_songs_2/*.mid")

In [67]:
all_midi_files = []
all_notes = []
for i in names[:5]:
    notes = []
    print (i)
    midi = converter.parse(i)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
            all_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
            all_notes.append('.'.join(str(n) for n in element.normalOrder))
    all_midi_files.append(notes)

/Users/luizeduardocartolano/Dropbox/DUDU/Unicamp/IC/MC906/workspace/Audio/midi_songs_2/Saint.mid
/Users/luizeduardocartolano/Dropbox/DUDU/Unicamp/IC/MC906/workspace/Audio/midi_songs_2/Thomas&TheMagicRailroad.mid
/Users/luizeduardocartolano/Dropbox/DUDU/Unicamp/IC/MC906/workspace/Audio/midi_songs_2/Bulletproof.mid
/Users/luizeduardocartolano/Dropbox/DUDU/Unicamp/IC/MC906/workspace/Audio/midi_songs_2/RobinHood-PrinceOfTheives.mid
/Users/luizeduardocartolano/Dropbox/DUDU/Unicamp/IC/MC906/workspace/Audio/midi_songs_2/CantBuyMeLove.mid


In [69]:
pitchnames = sorted(set(item for item in all_notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

In [70]:
all_midi_file_as_int = []
for i in all_midi_files:
    all_midi_file_as_int.append([note_to_int[char] for char in i])

In [73]:
only_histogram = []
for i in all_midi_file_as_int:
    np_array = np.asarray(i)
    hist, bin_edges = np.histogram(np_array, bins = len(pitchnames) , range = (0,len(pitchnames)), density=False)
    if (np.sum(hist) > 0):
        only_histogram.append(hist/np.sum(hist))

In [74]:
only_histogram = np.asarray(only_histogram)

In [80]:
only_histogram[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00079618,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00079618, 0.00079618, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00079618, 0.00079618,
       0.00079618, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.43471338, 0.00477707, 0.        , 0.00398089,
       0.01353503, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.2133758 , 0.00398089, 0.        , 0.00079

In [16]:
notes = []
midi = converter.parse(audio_teste)

In [17]:
notes_to_parse = None
parts = instrument.partitionByInstrument(midi)

In [54]:
if parts: # file has instrument parts
    notes_to_parse = parts.parts[0].recurse()
else: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

In [55]:
for element in notes_to_parse:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

In [58]:
def get_notes(path_to_song):
    """
        :param: path_to_song - path to a .mid file
        :return: notes - a list with all the notes extracted from the .mid file
    """
    notes = []
    midi = converter.parse(audio_teste)
    
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    # file has instrument parts
    if parts:
        notes_to_parse = parts.parts[0].recurse()
    # file has notes in a flat structure
    else: 
        notes_to_parse = midi.flat.notes
        
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
    
    return notes

In [60]:
# get all pitch names
pitchnames = sorted(set(item for item in notes))

In [62]:
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

In [69]:
network_input = []
network_input.append([note_to_int[char] for char in notes])

In [70]:
network_input

[[58,
  6,
  37,
  46,
  60,
  60,
  60,
  27,
  27,
  46,
  27,
  46,
  49,
  27,
  49,
  63,
  63,
  27,
  49,
  63,
  63,
  27,
  27,
  49,
  27,
  27,
  45,
  53,
  45,
  59,
  59,
  53,
  6,
  27,
  35,
  6,
  35,
  27,
  49,
  1,
  27,
  34,
  27,
  63,
  63,
  27,
  27,
  49,
  37,
  37,
  27,
  46,
  27,
  56,
  60,
  60,
  27,
  46,
  27,
  46,
  27,
  46,
  49,
  27,
  49,
  59,
  64,
  64,
  27,
  49,
  27,
  27,
  47,
  47,
  49,
  27,
  38,
  38,
  64,
  64,
  26,
  4,
  56,
  60,
  60,
  27,
  6,
  53,
  35,
  35,
  6,
  49,
  26,
  49,
  60,
  0,
  49,
  60,
  40,
  50,
  49,
  50,
  46,
  47,
  42,
  41,
  45,
  46,
  47,
  64,
  63,
  53,
  52,
  53,
  54,
  53,
  60,
  54,
  53,
  50,
  49,
  46,
  45,
  47,
  47,
  41,
  37,
  47,
  59,
  50,
  40,
  40,
  49,
  49,
  46,
  46,
  41,
  41,
  45,
  46,
  45,
  46,
  63,
  63,
  53,
  52,
  53,
  52,
  53,
  53,
  53,
  60,
  53,
  60,
  53,
  53,
  49,
  49,
  46,
  45,
  46,
  45,
  41,
  41,
  37,
  37,
  59,
  59,
